Ceneo Scraper

## Ekstracja składowych pojedynczej opinii
|składowa|Selektor|Zmienna|
|--------|--------|-------|
|id opinii|["data-entry-id"]|opinion_id
|autor |span.user-post__author-name|author
|rekomendacja |span.user-post__author-recomendation > em|recommendation
|gwiazdki |span.user-post__score-count|rating
|treść |div.user-post__text|content
|lista zalet |div.review-feature__title--positives ~ div.review-feature__item|pros
|lista wad |div.review-feature__title--negatives ~ div.review-feature__item|cons
|dla ilu przydatna |span[id^="votes-yes"]|useful
|dla ilu nieprzydatna |span[id^="votes-no"]|useless
|data wystawienia |span.user-post__published > time:nth-child(1)["datetime"]|post_date
|data zakupu |span.user-post__published > time:nth-child(2)["datetime"]|purchase_date

## biblioteki

In [1]:
import json
import os
import requests
from bs4 import BeautifulSoup

## Wysyłanie żądania dostępu do zasobu do serwera

In [2]:
product_id = "114700014" 
url= f"https://www.ceneo.pl/{product_id}/opinie-1"
#response = requests.get(url) kopiujemy do petli dla kazdego z osobna
#response.status_code

In [3]:
all_opinions = []
while (url):
    print(url)
    response = requests.get(url)
    page_dom = BeautifulSoup(response.text, "html.parser")
#print(type(page_dom))
    opinions = page_dom.select("div.js_product-review")
#print(type(opinions))
#opinion=opinions.pop(0)
#print(type(opinion))
#print(opinion)
    for opinion in opinions:
        try:
            single_opinion = {
                "opinion_id" : opinion["data-entry-id"],
                "author": opinion.select_one("span.user-post__author-name").get_text().strip(),
                "recommendation": opinion.select_one("span.user-post__author-recomendation > em").get_text().strip(),
                "rating": opinion.select_one("span.user-post__score-count").get_text().strip(),
                "content": opinion.select_one("div.user-post__text").get_text().strip(),
                "pros": [p.get_text().strip() for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
                "cons": [c.get_text().strip() for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
                "useful" : opinion.select_one("span[id^='votes-yes']").get_text().strip(),
              "useless": opinion.select_one("span[id^='votes-no']").get_text().strip(),
              "post_date": opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip(),
              "purchase_date": opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"].strip(),
            }
            all_opinions.append(single_opinion)
        except (AttributeError,TypeError):
            pass
    try:
        url = "https://www.ceneo.pl"+page_dom.select_one("a.pagination__next")["href"]
    except TypeError:
        url = None

https://www.ceneo.pl/114700014/opinie-1
https://www.ceneo.pl/114700014/opinie-2
https://www.ceneo.pl/114700014/opinie-3
https://www.ceneo.pl/114700014/opinie-4
https://www.ceneo.pl/114700014/opinie-5
https://www.ceneo.pl/114700014/opinie-6
https://www.ceneo.pl/114700014/opinie-7
https://www.ceneo.pl/114700014/opinie-8
https://www.ceneo.pl/114700014/opinie-9
https://www.ceneo.pl/114700014/opinie-10
https://www.ceneo.pl/114700014/opinie-11
https://www.ceneo.pl/114700014/opinie-12
https://www.ceneo.pl/114700014/opinie-13
https://www.ceneo.pl/114700014/opinie-14
https://www.ceneo.pl/114700014/opinie-15
https://www.ceneo.pl/114700014/opinie-16
https://www.ceneo.pl/114700014/opinie-17
https://www.ceneo.pl/114700014/opinie-18


In [ ]:

single_opinion = {
"opinion_id" : opinion["data-entry-id"],
"author": opinion.select_one("span.user-post__author-name").get_text().strip(),
"recommendation": opinion.select_one("span.user-post__author-recomendation > em").get_text().strip(),
"rating": opinion.select_one("span.user-post__score-count").get_text().strip(),
"content": opinion.select_one("div.user-post__text").get_text().strip(),
"pros": [p.get_text().strip() for p in opinion.select("div.review-feature__title--positives ~ div.review-feature__item")],
"cons": [c.get_text().strip() for c in opinion.select("div.review-feature__title--negatives ~ div.review-feature__item")],
"useful" : opinion.select_one("span[id^='votes-yes']").get_text().strip(),
"useless": opinion.select_one("span[id^='votes-no']").get_text().strip(),
"post_date": opinion.select_one("span.user-post__published > time:nth-child(1)")["datetime"].strip(),
"purchase_date": opinion.select_one("span.user-post__published > time:nth-child(2)")["datetime"].strip(),
}

In [4]:
#print(single_opinion)
if not os.path.exists("opinions"):
    os.mkdir("opinions")

#print(json.dumps(all_opinions, indent=4, ensure_ascii=False))
with open(f"opinions/{product_id}.json", "w", encoding="UTF-8") as jf:
    json.dump(all_opinions, jf, indent=4, ensure_ascii=False)

In [ ]:
len(opinions)